In [23]:
# Instalar pandas si no está instalado 

%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [24]:
# Import 

import pandas as pd

In [25]:
# Utils

def show_data_summary(data):
    print(f'Size: {data.size}')
    print(f'Columns: {len(data.columns)}')
    print(f'Rows: {data.shape[0]}')
    print(f'Unique clients: {len(data.client_id.unique())}')

In [26]:
# Load data

data = pd.read_csv('./data.csv', delimiter='|', skipfooter=1, engine='python')

show_data_summary(data)

Size: 18373355
Columns: 77
Rows: 238615
Unique clients: 26560


In [27]:
# Remove summary row

data = data[(data.client_id != '(238615 rows affected)')]

show_data_summary(data)

Size: 18373355
Columns: 77
Rows: 238615
Unique clients: 26560


In [28]:
# Remove duplicates

data = data.drop_duplicates(subset=['Month', 'client_id'])

show_data_summary(data)

Size: 18373355
Columns: 77
Rows: 238615
Unique clients: 26560


In [29]:
# Get clients with 9 months data

nine_mouths = data.groupby('client_id')['Month'].count().reset_index()
clients_with_9_months = data.merge(
    nine_mouths[nine_mouths.Month == 9][['client_id']],
    how='inner',
    on='client_id',
)

show_data_summary(clients_with_9_months)

Size: 18352719
Columns: 77
Rows: 238347
Unique clients: 26483


##### Windows

- Training window: 6 month (from 2018-11-01 to 2019-01-01)
- Lead window: 1 month (2019-02-01)
- Prediction window: last 2 month (2019-03-01 and 2019-04-01)

In [30]:
# Get last training month clients without cobranding

last_training_month = '2019-01-01'
last_training_month_data = clients_with_9_months[clients_with_9_months.Month == last_training_month]

clients_without_cobranding = clients_with_9_months.merge(
    last_training_month_data[last_training_month_data.CreditCard_CoBranding == 'No'][['client_id']],
    how='inner',
    on='client_id',
)

show_data_summary(clients_without_cobranding)

Size: 16386678
Columns: 77
Rows: 212814
Unique clients: 23646


In [31]:
last_training_month_data = clients_without_cobranding[clients_without_cobranding.Month == last_training_month]
last_training_month_data.Package_Active.value_counts()

Package_Active
No     23191
Yes      455
Name: count, dtype: int64

In [32]:
# Get last training month clients without package active

last_training_month_data = clients_without_cobranding[clients_without_cobranding.Month == last_training_month]

clients_without_cobranding_without_package = clients_without_cobranding.merge(
    last_training_month_data[last_training_month_data.Package_Active == 'No'][['client_id']],
    how='inner',
    on='client_id',
)

show_data_summary(clients_without_cobranding_without_package)

Size: 16071363
Columns: 77
Rows: 208719
Unique clients: 23191
